In [1]:
import os
import json
import time
import requests
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# === xAI Grok API setup ===
api_key = "api-key"  # 🔒 Replace with your secure token
api_url = "https://api.x.ai/v1/chat/completions"
model_name = "grok-3-mini"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# === Paths ===
base = Path.home() / "Desktop" / "Benchmark"
subs_dir = base / "json_benchmark"
summaries_dir = base / "summaries"
output_dir = base / "scored_themes_grok"
prompts_dir = base / "prompts_themes_grok"

output_dir.mkdir(parents=True, exist_ok=True)
prompts_dir.mkdir(parents=True, exist_ok=True)

# === Load match file ===
matches_df = pd.read_csv("matches_benchmark.csv")

# === Prompt template ===
def create_theme_prompt(summary, chunk):
    prompt = (
        "You are analyzing a film based on its dialogue and a summary of its plot.\n\n"
        "Your task is to identify the ideological, societal, or political themes expressed in the film.\n\n"
        "Definitions:\n"
        "- Ideological themes refer to ideas or values associated with a worldview or belief system, such as attitudes toward authority, individual rights, or tradition.\n"
        "- Societal themes reflect cultural norms, social roles, or commentary on how people relate to institutions, communities, or each other.\n"
        "- Political themes involve power structures, governance, laws, rights, or critiques of political institutions or ideologies.\n\n"
        "Please:\n"
        "1. Provide a list of 3 to 7 themes in the form of keywords or short phrases.\n"
        "2. Include a confidence score from 0.0 to 1.0 indicating how confident you are in your analysis.\n"
        "3. Provide a short explanation (1–2 plain sentences, no new lines or bullet points).\n\n"
        "Return your answer strictly in the following JSON format:\n\n"
        "{\n"
        "  \"themes\": [\"...\", \"...\", \"...\"],\n"
        "  \"confidence\": ...,\n"
        "  \"explanation\": \"...\"\n"
        "}\n\n"
        "All fields must be JSON-safe and use single-line values only. Do not use line breaks, markdown, or bullet points.\n\n"
        f"Film Summary:\n\"\"\"{summary}\"\"\"\n\n"
        f"Dialogue:\n\"\"\"{chunk}\"\"\""
    )
    return prompt

# === API call to Grok ===
def call_grok(prompt):
    payload = {
        "model": model_name,
        "temperature": 0,
        "stream": False,
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are a careful film analyst. Return only compact, strictly valid JSON with no formatting, "
                    "no line breaks, and no markdown. Your explanation field must be a single line of plain text."
                )
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    response = requests.post(api_url, headers=headers, json=payload)

    if response.status_code == 429:
        print("⚠️ Rate limit hit. Waiting...")
        time.sleep(60)
        return call_grok(prompt)
    elif response.status_code != 200:
        print(f"❌ API Error: {response.status_code}")
        print(response.text)
        return None

    try:
        content = response.json()["choices"][0]["message"]["content"]
        return json.loads(content)
    except Exception as e:
        print(f"⚠️ JSON parsing error:\n{response.text}")
        return None

# === Subtitle chunking ===
def chunk_dialogue(subs, chunk_size=5000, overlap=200):
    text_blocks = [line.get("text", "").strip() for line in subs if line.get("text")]
    full_text = " ".join(text_blocks)

    chunks = []
    start = 0
    while start < len(full_text):
        end = start + chunk_size
        chunks.append(full_text[start:end])
        start = end - overlap
    return chunks

# === Main loop ===
for _, row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Scoring themes with Grok"):
    filename = row["subtitle_filename"]
    json_path = subs_dir / f"{filename}.json"
    summary_path = summaries_dir / f"{filename}.srt_summary.txt"

    if not json_path.exists() or not summary_path.exists():
        print(f"⚠️ Missing files for: {filename}")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        subs = json.load(f)

    with open(summary_path, "r", encoding="utf-8") as f:
        summary = f.read().strip()

    chunks = chunk_dialogue(subs)

    for i, chunk in enumerate(chunks):
        output_path = output_dir / f"{filename}_chunk{i+1}_themes.json"
        prompt_path = prompts_dir / f"{filename}_chunk{i+1}_themes_prompt.txt"

        if output_path.exists():
            print(f"🟡 Already scored: {filename} chunk {i+1}")
            continue

        prompt = create_theme_prompt(summary, chunk)

        with open(prompt_path, "w", encoding="utf-8") as f:
            f.write(prompt)

        result = call_grok(prompt)
        if result:
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(result, f, indent=2)
            print(f"✅ Scored: {filename} chunk {i+1}")
        else:
            print(f"❌ Failed: {filename} chunk {i+1}")

        time.sleep(1.5)

Scoring themes with Grok:   0%|                          | 0/20 [00:00<?, ?it/s]

✅ Scored: 2006Blood.Diamond chunk 1
✅ Scored: 2006Blood.Diamond chunk 2
✅ Scored: 2006Blood.Diamond chunk 3
✅ Scored: 2006Blood.Diamond chunk 4
✅ Scored: 2006Blood.Diamond chunk 5
✅ Scored: 2006Blood.Diamond chunk 6
✅ Scored: 2006Blood.Diamond chunk 7
✅ Scored: 2006Blood.Diamond chunk 8
✅ Scored: 2006Blood.Diamond chunk 9
✅ Scored: 2006Blood.Diamond chunk 10
✅ Scored: 2006Blood.Diamond chunk 11


Scoring themes with Grok:   5%|▉                 | 1/20 [01:08<21:44, 68.65s/it]

✅ Scored: 2005The.Constant.Gardener chunk 1
✅ Scored: 2005The.Constant.Gardener chunk 2
✅ Scored: 2005The.Constant.Gardener chunk 3
✅ Scored: 2005The.Constant.Gardener chunk 4
✅ Scored: 2005The.Constant.Gardener chunk 5
✅ Scored: 2005The.Constant.Gardener chunk 6
✅ Scored: 2005The.Constant.Gardener chunk 7
✅ Scored: 2005The.Constant.Gardener chunk 8
✅ Scored: 2005The.Constant.Gardener chunk 9
✅ Scored: 2005The.Constant.Gardener chunk 10
✅ Scored: 2005The.Constant.Gardener chunk 11
✅ Scored: 2005The.Constant.Gardener chunk 12
✅ Scored: 2005The.Constant.Gardener chunk 13


Scoring themes with Grok:  10%|█▊                | 2/20 [02:26<22:09, 73.87s/it]

...

Scoring themes with Grok:  90%|███████████████▎ | 18/20 [19:50<02:28, 74.48s/it]

✅ Scored: 1985Back.To.The.Future chunk 1
✅ Scored: 1985Back.To.The.Future chunk 2
✅ Scored: 1985Back.To.The.Future chunk 3
✅ Scored: 1985Back.To.The.Future chunk 4
✅ Scored: 1985Back.To.The.Future chunk 5
✅ Scored: 1985Back.To.The.Future chunk 6
✅ Scored: 1985Back.To.The.Future chunk 7
✅ Scored: 1985Back.To.The.Future chunk 8
✅ Scored: 1985Back.To.The.Future chunk 9
✅ Scored: 1985Back.To.The.Future chunk 10
✅ Scored: 1985Back.To.The.Future chunk 11


Scoring themes with Grok:  95%|████████████████▏| 19/20 [20:57<01:12, 72.17s/it]

✅ Scored: 2013The.Purge chunk 1
✅ Scored: 2013The.Purge chunk 2
✅ Scored: 2013The.Purge chunk 3
✅ Scored: 2013The.Purge chunk 4
✅ Scored: 2013The.Purge chunk 5
✅ Scored: 2013The.Purge chunk 6
✅ Scored: 2013The.Purge chunk 7


Scoring themes with Grok: 100%|█████████████████| 20/20 [21:39<00:00, 65.00s/it]
